In [ ]:

#import plotly.graph_objects as go
from collections import defaultdict, Counter
from itertools import permutations, cycle, islice, chain
import os
from pathlib import Path

from scipy.sparse import *
from scipy.stats import wasserstein_distance

from sklearn.metrics import silhouette_samples, silhouette_score, accuracy_score, recall_score, precision_score,mean_squared_error, f1_score
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans, MiniBatchKMeans, Birch, DBSCAN, OPTICS
from sklearn.mixture import GaussianMixture
from sklearn.linear_model import LogisticRegression as LR
from sklearn.linear_model import Ridge
from sklearn.preprocessing import normalize
from sklearn.metrics import pairwise_distances
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

from gensim.models.doc2vec import TaggedDocument, Doc2Vec
from gensim.models.word2vec import Word2Vec

import matplotlib.colors as mcolors
import matplotlib.cm as cm

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import display

#from interpret import show
#from interpret.perf import ROC
#from interpret.glassbox import ExplainableBoostingClassifier, LogisticRegression, ClassificationTree, DecisionListClassifier
#from interpret.blackbox import ShapKernel
#import shap
#from xgboost import XGBClassifier
#from interpret.data import ClassHistogram

#import tensorflow as tf
#import tensorboard as tb
#tf.io.gfile = tb.compat.tensorflow_stub.io.gfile
from data_loader import *
from utils import *


#from torch.utils.tensorboard import SummaryWriter
sns.set()
%matplotlib inline
%load_ext autoreload
%autoreload 1
%aimport utils

In [ ]:
%%time
Dataset()
Users()
CityStats()

# Embeddings block

In [ ]:
%%time
LOAD_EMBEDDINGS = True
if LOAD_EMBEDDINGS and os.path.exists(EMBEDDING_PATH):
    with open(EMBEDDING_PATH, 'rb') as f:
        embeddings_dict = pickle.load(f)
else:
    dataset = [Dataset().id_to_transaction[t_id].to_sentence(barcodes) for t_id, barcodes in Dataset().barcode_iterator()]
    print('created_dataset')
    gensim_skipgram = Word2Vec(dataset,vector_size=300, window=100, min_count=1, workers=8,epochs=5,sample=1e-4, shrink_windows=False,) 
    with open(EMBEDDING_PATH, 'wb') as f:
        pickle.dump(gensim_skipgram.wv, f)
    embeddings_dict = gensim_skipgram.wv
    del dataset

products = sorted(Dataset().products, key=lambda x: x.usages, reverse=True)[:5000]
product_vectors = np.vstack([embeddings_dict[x.barcode] for x in products])

print(Dataset().barcode_to_product['7290004127329'].name)
for x, y in embeddings_dict.most_similar('7290004127329'):
    print(Dataset().barcode_to_product[x].name, y)

In [ ]:
to_show = 60
decompose(product_vectors[:300], 
          show=to_show, 
          hue=[smart_inverse(x.hierarchy_names[2]) for x in products[:to_show]], 
          annotations=[smart_inverse(x.name) for x in products[:to_show]])

# City stats block

In [ ]:
stats_df = pd.DataFrame(CityStats().stats_df)
stats_df = stats_df[stats_df.n_users > 30]
clalit_cities = pd.read_csv('./stats/cities_clalit.csv')
clalit_cities.columns=['city', 'district']
clalit_cities.city = clalit_cities.city.map(lambda x: convert_city_name(x))
city_to_region = {x:translate_district(y) for x,y in clalit_cities.values}
stats_df['district'] = stats_df.index.map(lambda x: city_to_region.get(x, 'Unknown'))
stats_df.head()

# Clalit data

In [ ]:
districts = """Jerusalem/1
Tel Aviv/2
Dan - Petach Tikva/4
Haifa/5
Central/7
South/9
Sharon - Shomron/12
North/15
Eilat/16""".split('\n')
districts = [x.split('/') for x in districts]
districts = {x[1]: x[0] for x in districts}

df = pd.read_csv('./stats/splits.csv')
df = df[df.condition == 'active_malignancy=0 and ibd=0 and rhemartoid_arthritis=0']
df = df[df.pop_name == 'overall']
df=df[df.test_name != 'znb']

clalit_splits = df[(df.first_split == 'sector') & (df.second_split == 'district')& (df.third_split == 'age_group')& (df.fourth_split.isna())]
clalit_splits['district'] = clalit_splits.second_split_val.apply(lambda x: districts[x])
clalit_splits=clalit_splits[clalit_splits.district != 'Eilat']
districts =  clalit_splits.district.unique() 

clalit_splits['sector'] = clalit_splits.first_split_val.apply(lambda x: {'1': 'Arab', '2': "Haredi", '9': 'Others'}[x])
clalit_splits = clalit_splits[clalit_splits.sector != 'Arab']
clalit_splits['age'] = clalit_splits['third_split_val'].apply(lambda x: '09-13' if x== '9-13' else x)

# Normal percentages for test/district/sector
norms = {}
for test in clalit_splits.test_name.unique():
    for district in clalit_splits.district.unique():
        for sector in clalit_splits.sector.unique():
            cs = clalit_splits[(clalit_splits.test_name == test) & (clalit_splits.district == district) & (clalit_splits.sector == sector)].set_index('age').normal_pct.to_dict()
            norms[(test,district, sector)] = cs

In [ ]:
clalit_splits = df[(df.first_split == 'sector') & (df.second_split == 'age_group')& (df.third_split.isna())]
#clalit_splits['district'] = clalit_splits.second_split_val.apply(lambda x: districts[x])
clalit_splits['sector'] = clalit_splits.first_split_val.apply(lambda x: {'1': 'Arab', '2': "Haredi", '9': 'Others'}[x])
clalit_splits = clalit_splits[clalit_splits.sector != 'Arab']
clalit_splits['age'] = clalit_splits['second_split_val'].apply(lambda x: '09-13' if x== '9-13' else x)
# Normal percentages for test/sector
for test in clalit_splits.test_name.unique():
    for sector in clalit_splits.sector.unique():
        cs = clalit_splits[(clalit_splits.test_name == test) & (clalit_splits.sector == sector)].set_index('age').normal_pct.to_dict()
        norms[(test,sector)] = cs

In [ ]:

ages = sorted(clalit_splits.age.unique())[:8]
distances = {}
city_list = []
for row in stats_df.iterrows():
    city, series = row[0], row[1]
    if series.district == 'Unknown' or series.district == 'Eilat':
        continue
    city_list.append(city)
    ages_v = series.values[2:10]
    ages_v = ages_v/ages_v.sum()
    orig_district = series.district
    for test in clalit_splits.test_name.unique():
        for district in districts:
            distances[test, city, district] = 0
            for i, age in enumerate(ages):
                city_norm = norms[test,orig_district, 'Others'][age] if age in norms[test,orig_district, 'Others'] else norms[test, 'Others'][age]
                new_region_norm = norms[test,district, 'Others'][age] if age in norms[test,district, 'Others'] else norms[test, 'Others'][age]
                distances[test, city, district] += (1-series.haredim/100)*np.abs(city_norm - new_region_norm)*ages_v[i]
                
                city_norm = norms[test,orig_district, 'Haredi'][age] if age in norms[test,orig_district, 'Haredi'] else norms[test, 'Haredi'][age]
                new_region_norm = norms[test,district, 'Haredi'][age] if age in norms[test,district, 'Haredi'] else norms[test, 'Haredi'][age]
                distances[test, city, district] += series.haredim/100*np.abs(city_norm - new_region_norm)*ages_v[i]

city_list = sorted(city_list, key=lambda x: city_to_region[x])

# Food distance:
### L2 distance between embeddings

# Age distance:
### Wasserstein distance, also known as the earth mover’s distance, since it can be seen as the minimum amount of “work” required to transform $u$ into $v$, where “work” is measured as the amount of distribution weight that must be moved, multiplied by the distance it has to be moved.

# Health distance:

### <b>Assume we can "move" city A from district A to district B, completely replacing all health statistics. How many people will change their health status according to test T? </b>
#### $d(cityA,districtB) = \sum_{test \ T} \sum_{ages \ in \ A} \ (p^A_{age} \cdot |normal_T A - normal_T B|)$


### <b>Now imagine we swap districts of city A and B:</b>
#### $d(cityA,cityB) = d(cityA,distictB) + d(cityB,districtA)$


# Total Distance:
### We scale distances of each type to have average ~1, and then take weighted sum: $d(A,B)=w_1 \cdot d_{age} + w_2 \cdot d_{food} + w_3 \cdot d_{health}$

### <b>Problem: citites in the same district receive distance = 0</b>
#### Current solution: for cities in the same district we set heatlh distance as average between food and age distances

In [ ]:

health_distances = np.zeros((len(city_list), len(city_list)))
for i, c1 in enumerate(city_list):
    d1 = city_to_region[c1]
    
    for j, c2 in enumerate(city_list):
        d2 = city_to_region[c2]
        if d1 == d2:
            health_distances[i,j] = 0
            continue
        distance = 0
        for test in clalit_splits.test_name.unique():
            distance += distances[test, c1, d2]
            distance += distances[test, c2, d1]
        health_distances[i,j] = distance
mean = health_distances.mean()
health_distances /= mean
health_distances.shape

In [ ]:

ages = CityStats().population_df.loc[city_list].values[:, 2:68].astype(float)
totals = ages.sum(axis=1)

ages = np.divide(ages.T,totals).T
age_distances = pairwise_distances(ages, metric=wasserstein_distance)
age_distances /= age_distances.mean()
mean_ages = ages.dot(np.array(list(range(66))))

city_embeddings = np.vstack([embeddings_dict[x] for x in city_list])
city_embeddings_normalized = normalize(city_embeddings, axis=1)
food_distances = 2-2*city_embeddings_normalized.dot(city_embeddings_normalized.T)

food_distances *= (food_distances > 0)
food_distances.shape, food_distances.min()

In [ ]:
def build_heatmaps():
    regions = sorted({city_to_region[x] for x in city_list})
    health_distances_reg = np.zeros((len(city_list), len(regions)))
    for i, c1 in enumerate(city_list):
        for j, d2 in enumerate(regions):
            d1 = city_to_region[c1]
            if d1 == d2:
                health_distances_reg[i,j] = 0
                continue
            distance = 0
            for test in clalit_splits.test_name.unique():
                distance += distances[test, c1, d2]
            health_distances_reg[i,j] = distance
    mean = health_distances_reg.mean()
    health_distances_reg /= mean
    plt.figure(figsize=(5,20))
    df = pd.DataFrame(health_distances_reg.round(1), columns=regions, index=[smart_inverse(x) for x in city_list])
    sns.heatmap(df, annot=True)
    plt.savefig('./img/Health_regions.png')
    plt.figure(figsize=(20,20))
    df = pd.DataFrame(health_distances.round(1), columns=[smart_inverse(x) for x in city_list], index=[smart_inverse(x) for x in city_list])
    p = sns.heatmap(df, annot=True)
    p.set_title('Health')
    plt.savefig('./img/health.png')
    plt.figure(figsize=(20,20))
    df = pd.DataFrame(age_distances.round(1), columns=[smart_inverse(x) for x in city_list], index=[smart_inverse(x) for x in city_list])
    p = sns.heatmap(df, annot=True)
    p.set_title('Age')
    plt.savefig('./img/age.png')
    plt.figure(figsize=(20,20))
    df = pd.DataFrame(food_distances.round(1), columns=[smart_inverse(x) for x in city_list], index=[smart_inverse(x) for x in city_list])
    p = sns.heatmap(df, annot=True)
    p.set_title('Food')
    plt.savefig('./img/food.png')

In [ ]:
def get_distance(food_weight=1, age_weight=0, health_weight=2):
    total_distance = np.zeros((len(city_list), len(city_list)))

    for i, c1 in enumerate(city_list):
        d1 = city_to_region[c1]
        for j, c2 in enumerate(city_list):
            d2 = city_to_region[c2]
            if i == j:
                total_distance[i,j] = 0
            elif d1 == d2:
                total_distance[i,j] = food_distances[i,j]*(food_weight+health_weight/2)+age_distances[i,j]*(age_weight+health_weight/2)
            else:
                total_distance[i,j] = food_distances[i,j]*food_weight+age_distances[i,j]*age_weight + health_distances[i,j]*health_weight
    return total_distance

In [ ]:

p = decompose(get_distance(0,1,0), init='random', metric='precomputed', title=f'by age', hue=mean_ages, annotations=[smart_inverse(x) for x in city_list], perplexity=20, learning_rate=50, ee=20)
for food, age, health in [(1,0,0),(1,0,1), (1,1,1), (1,1,5), (5,1,1)]:
    total_distance = get_distance(food,age,health)
    p = decompose(total_distance, init='random', metric='precomputed', title=f'by combination. Food importance: {food}. Age importance: {age}. Health importance: {health}. ', hue=stats_df.loc[city_list].district, annotations=[smart_inverse(x) for x in city_list], perplexity=20, learning_rate=50, ee=20)
    p.figure.savefig(f'./img/Mixed map {food} {age} {health}.png')

# UI

In [ ]:
%%time
LOAD_VECS = True
if LOAD_VECS and os.path.exists(CITY_PRODUCT_COUNTS_PATH):
    with open(CITY_PRODUCT_COUNTS_PATH, 'rb') as f:
        city_vectors = pickle.load(f)
else:
    city_vectors = {c: np.zeros(sum(Dataset().hierarchy_sizes)) for c in city_list}
    city_set = set(city_list)
    for t_id, barcodes in Dataset().barcode_iterator():
        t = Dataset().id_to_transaction[t_id]
        city = Users().active_users_cities.get(t.user_id, 'none')
        if city not in city_set:
            continue
        #user_vectors[t.user_id][[Dataset().barcode_to_product[barcode].hierarchy_indices[3]-offset for barcode in t.barcodes]] += 1
        for barcode in barcodes:
            product = Dataset().barcode_to_product[barcode]
            city_vectors[city][product.hierarchy_indices] += 1
    with open(CITY_PRODUCT_COUNTS_PATH, 'wb') as f:
        pickle.dump(city_vectors, f)

index_to_name = {v:k for k,v in Dataset().index_of.items()}

In [ ]:

clusters = {}
clusters['north'] = stats_df[stats_df.district == 'North'].index.values.tolist()
clusters['south'] = stats_df[stats_df.district == 'South'].index.values.tolist()
clusters['haifa'] = stats_df[stats_df.district == 'Haifa'].index.values.tolist()
clusters['religion'] = stats_df[stats_df.haredim > 60 ].index.values.tolist()
clusters['jerusalem'] = stats_df[(stats_df.district == 'Jerusalem') & (stats_df.haredim < 60)].index.values.tolist()
clusters['jerusalem_and_religion'] =  stats_df[(stats_df.district == 'Jerusalem') | (stats_df.haredim > 60)].index.values.tolist()
clusters['sharon'] = [x for x in stats_df[stats_df.district == 'Sharon - Shomron'].index.values.tolist() if x not in ["רמת השרון", "כפר סבא", "רעננה", "הרצלייה"]]
#clusters['sharon'] = [x for x in sharon]
#tel_aviv = ["חולון", "בת ים", "ראשון לציון"]
clusters['center_dan'] = [x for x in city_list if x not in chain.from_iterable(clusters.values())]

city_to_cluster = {}
for k,v in clusters.items():
    for city in v:
        city_to_cluster[city] = k
        
for city in sorted(city_list):
    clusters[city] = [city]

In [ ]:
for food, age, health in [(3,0,1)]:
    total_distance = get_distance(food,age,health)
    p = decompose(total_distance, init='random', metric='precomputed', title=f'by combination. Food importance: {food}. Age importance: {age}. Health importance: {health}. ', hue=stats_df.loc[city_list].district, annotations=[smart_inverse(x) for x in city_list], perplexity=20, learning_rate=50, ee=50)
    p = decompose(total_distance, init='random', metric='precomputed', title=f'by combination. Food importance: {food}. Age importance: {age}. Health importance: {health}. ', 
                  hue=[city_to_cluster[x] for x in city_list], annotations=[smart_inverse(x) for x in city_list], perplexity=20, learning_rate=50, ee=50)

In [ ]:
from ipywidgets import HBox, VBox, interact, Layout
d = []
cluster1 = widgets.Dropdown(options=list(clusters.keys()),value='jerusalem_and_religion',description='Cluster 1:',disabled=False)
cluster2 = widgets.Dropdown(options=['not cluster 1'] + list(clusters.keys()) ,value='not cluster 1',description='Cluster 2:',disabled=False)
level = widgets.Dropdown(options=['CLASS', 'CATEGORY', 'SUB-CATEGORY', 'BARCODE'] ,value='SUB-CATEGORY',description='Level:',disabled=False)
number_1 = widgets.IntSlider(min=10, max=50, step=5, value=15, description='# features +')
number_2 = widgets.IntSlider(min=0, max=30, step=5, value=15,description='# features -')

food_weight = widgets.IntSlider(min=0, max=5, step=1, value=3,description='Food importance', style = {'description_width': 'initial'})
age_weight = widgets.IntSlider(min=0, max=5, step=1, value=0,description='Age importance', style = {'description_width': 'initial'})
health_weight = widgets.IntSlider(min=0, max=5, step=1, value=1,description='Health importance', style = {'description_width': 'initial'})

ui_weights = HBox([food_weight, age_weight, health_weight])
ui_clusters = HBox([cluster1, cluster2])
ui_explanation = HBox([level, number_1, number_2])
ui = VBox([ui_weights, ui_clusters, ui_explanation])

transformed_X = None
vizualization_weights = (0,0,0)

def show_food_difference_embeddings(city_list1, city_list2, number_1, number_2):
    vector1 = np.vstack([embeddings_dict[x] for x in city_list1]).mean(axis=0)
    vector2 = np.vstack([embeddings_dict[x] for x in city_list2]).mean(axis=0)
    difference_vector = vector1 - vector2
    items = embeddings_dict.most_similar(difference_vector, topn=1000)
    items = [(Dataset().barcode_to_product[x[0]].name, min(x[1], 1)) for x in items if x[0] in Dataset().barcode_to_product][:number_1]
    neg_items = embeddings_dict.most_similar(-difference_vector, topn=1000)
    items += [(Dataset().barcode_to_product[x[0]].name, -x[1]) for x in neg_items if x[0] in Dataset().barcode_to_product][:number_2]
    items = [(smart_inverse(x[0]), x[1]) for x in items]
    df = pd.DataFrame(items, columns=['name', 'similarity'])
    plt.figure(figsize=(15, len(items)/3))
    sns.barplot(x='similarity', y='name', data=df)
    
def on_update(cluster1, cluster2, level, number_1, number_2):
    root = Path(f'./img/clusters/{cluster1} - {cluster2}/')
    os.makedirs(root, exist_ok=True)
    city_list1 = clusters[cluster1]
    if len(city_list1) == 1:
        cluster1 = cluster1[::-1]
        
    if cluster2 == 'not cluster 1':
        city_list2 = [x for x in city_list if x not in city_list1]
        cluster2 = f'not {cluster1}'
    else:
        city_list2 = clusters[cluster2]
    
    df1 = stats_df.loc[city_list1]
    df2 = stats_df.loc[city_list2]
    
    text =  f'{cluster1}       |      {cluster2} \n'
    text += f'Population:       {df1.population.sum()//1000}K      |      {df2.population.sum()//1000}K \n'
    text += f'Users:           {df1.n_users.sum()}      |      {df2.n_users.sum()} \n'
    text += f'Socioeconomic: {(df1.socio*df1.n_users).sum()/df1.n_users.sum():.2f}      |      {(df2.socio*df2.n_users).sum()/df2.n_users.sum():.2f} \n'
    text += f'Periphery:    {(df1.periphery*df1.n_users).sum()/df1.n_users.sum():.2f}      |      {(df2.periphery*df2.n_users).sum()/df2.n_users.sum():.2f} \n'
    text += f'Avg salary:    {(df1.salary*df1.n_users).sum()/df1.n_users.sum():.0f}      |      {(df2.salary*df2.n_users).sum()/df2.n_users.sum():.0f} \n'
    f = plt.figure(figsize=(10,5))
    ax = f.add_subplot(111)
    plt.text(0.5,0.5,text,horizontalalignment='center', verticalalignment='center', transform = ax.transAxes, fontsize=20)
    ax.grid(False)
    ax.set_xticks([])
    ax.set_yticks([])
    plt.savefig(root / '0_stats.png')
    
    # combined 2d map    
    global transformed_X, vizualization_weights
    if vizualization_weights != (food_weight.value,age_weight.value,health_weight.value): # checking if weights changed
        vizualization_weights = (food_weight.value,age_weight.value,health_weight.value)
        transformed_X = None
        
    hue = [cluster1 if city in city_list1 else (cluster2 if city in city_list2 else 'no cluster') for city in city_list]
    p, transformed_X = decompose(get_distance(food_weight.value,age_weight.value,health_weight.value), init='random', metric='precomputed', 
                                 title=f'by combination. Food importance: {food_weight.value}. Age importance: {age_weight.value}. Health importance: {health_weight.value}. ', 
                                 hue=hue, hue_order=[cluster1, cluster2, 'no cluster'], 
                                 annotations=[smart_inverse(x) for x in city_list], 
                                 perplexity=20, learning_rate=50, ee=50, figsize=(15,10), 
                                 return_coordinates=True, transformed_X=transformed_X)
    plt.savefig(root / '1_map.png')
    
    # food difference
    level = ['CLASS', 'CATEGORY', 'SUB-CATEGORY', 'BARCODE'].index(level) + 1
    start_index = sum(Dataset().hierarchy_sizes[:level])
    end_index = start_index + Dataset().hierarchy_sizes[level]
    vector1 = normalize(np.sum(np.vstack([city_vectors[x][start_index:end_index] for x in city_list1]),axis=0).reshape(1,-1), norm='l1')[0]
    vector2 = normalize(np.sum(np.vstack([city_vectors[x][start_index:end_index] for x in city_list2]),axis=0).reshape(1,-1), norm='l1')[0]
    diff = vector1-vector2
    positive_idx = np.argpartition(diff, -number_1)[-number_1:]
    items = [(smart_inverse(index_to_name[i+start_index]), diff[i]*100) for i in positive_idx]
    negative_idx = np.argpartition(diff, number_2)[:number_2]
    items += [(smart_inverse(index_to_name[i+start_index]), diff[i]*100) for i in negative_idx]
    df = pd.DataFrame(items, columns=['name', 'frequency(%)']).sort_values('frequency(%)',ascending=False)
    #df['name'] = ['item' + str(x) for x in range(len(df))]
    plt.figure(figsize=(15, len(items)/3))
    p = sns.barplot(x='frequency(%)', y='name', data=df)
    p.set_title(f'Food frequency (right - {cluster1}')
    plt.savefig(root / 'food_frequency.png')
    
    # Age distribution
    plt.figure(figsize=(10,7))    
    ages = CityStats().population_df.loc[city_list].values[:, 2:68].astype(float)
    ages = np.divide(ages.T,ages.sum(axis=1)).T
    index1 = [i for i, x in enumerate(city_list) if x in city_list1]
    index2 = [i for i, x in enumerate(city_list) if x in city_list2]
    df = pd.DataFrame(ages, columns=list(range(66)))
    df['cluster'] = ['1' if x in city_list1 else '2' for x in city_list]
    p=sns.lineplot(y=ages[index1].mean(axis=0), x=list(range(66)))
    p=sns.lineplot(y=ages[index2].mean(axis=0), x=list(range(66)))
    plt.legend([cluster1, cluster2])
    plt.title('Age distribution')
    plt.savefig(root / '3_age_distribution.png')
    
    # health differences
    test_distances = defaultdict(int)
    for test in clalit_splits.test_name.unique():
        for i, c1 in enumerate(city_list1):
            d1 = city_to_region[c1]
            for j, c2 in enumerate(city_list2):
                d2 = city_to_region[c2]
                test_distances[test] += distances[test, c1, d2]
                test_distances[test] += distances[test, c2, d1]
    df = pd.DataFrame(test_distances.items(), columns=['test', 'difference']).sort_values('difference')
    plt.figure(figsize=(10,5))
    sns.barplot(x='difference', y='test', data=df)
    plt.savefig(root / '4_health_1.png')
    
    ages = sorted(clalit_splits.age.unique())[:8]
    f, ax = plt.subplots(2,2, figsize=(12,12))
    for i, test in enumerate(df.test.values[-4:]):
        data = defaultdict(int)
        for city in city_list1 + city_list2:
            district = city_to_region[city]
            
            religion_rate = stats_df.loc[city].haredim
            population_percentage = stats_df.loc[city].population
            population_percentage /= (stats_df.loc[city_list1].population.sum() if city in city_list1 else stats_df.loc[city_list2].population.sum())
            for age in ages:
                city_norm = (norms[test,district, 'Others'][age] if age in norms[test,district, 'Others'] else norms[test, 'Others'][age])*(1-religion_rate/100)
                city_norm += (norms[test,district, 'Haredi'][age] if age in norms[test,district, 'Haredi'] else norms[test, 'Haredi'][age])*religion_rate/100
                data[age, [cluster1, cluster2][city in city_list1]] += city_norm*population_percentage
        df = pd.DataFrame([list(k)+[v] for k,v in data.items()], columns=['age', 'cluster', 'normal percentage'])
        p = sns.lineplot(x='age', y='normal percentage', hue='cluster', hue_order=[cluster1, cluster2], ax=ax[i//2][i%2], data=df)
        p.set_title(test)
    plt.savefig(root / '4_health_2.png')
    
    
#interact(on_button_clicked,cluster1=cluster1, cluster2=cluster2);
out = widgets.interactive_output(on_update, {'cluster1': cluster1, 'cluster2': cluster2, 'level': level, 'number_1': number_1, 'number_2': number_2})

display(ui, out)

# TODO:
1) city-level clusters (text box or something)
2) add category food graph (religion: + barcode food graph)
3) prepare 8 dashboard screenshots (one for each cluster)
4) Check exactly how socio and periphery value
5) Add health distribution graph: top-4 tests with distribution curves (one grapoh for each test)

In [ ]:
MIN_DIFF = 1
def _region_pairwise_rank():
    for r1, v1 in ranks.items():
        for r2, v2 in ranks.items():
            vec1 = city_embeddings[[x==r1 for x in region_list]].mean(axis=0)
            vec2 = city_embeddings[[x==r2 for x in region_list]].mean(axis=0)
            comparison_X.append(vec1-vec2)
            if np.abs(v1-v2) < MIN_DIFF:
                comparison_y.append(0)
                comparison_X.append(vec1-vec2)
                comparison_y.append(1)
                continue
            comparison_y.append(v1 < v2)
def fit(comparison_X, comparison_y):
    model = LR(C=0.01, max_iter=10000)
    model.fit(comparison_X, comparison_y)
    top_products = [x.barcode for x in Dataset().products if x.usages > 10000]
    return [[Dataset().barcode_to_product[x].name, x, (embeddings_dict[x].dot(model.coef_[0]) - model.intercept_)[0]] for x in top_products]
    
def importance_from_order(order, name):
    order = order.split(' ')
    ranks = {}
    for region in set(user_regions):
        rank = [4-i for i,x in enumerate(order) if x.find(region.split(' ')[0]) != -1][0]
        ranks[region] = rank
    comparison_X = []
    comparison_y = []
    for i, city1 in enumerate(city_list):
        for j, city2 in enumerate(city_list):
            if city1 == city2:
                continue
            vec1 = city_embeddings[i]
            vec2 = city_embeddings[j]
            comparison_X.append(vec1-vec2)
            v1, v2 = ranks[city_to_region[city1]], ranks[city_to_region[city2]]

            if np.abs(v1-v2) < MIN_DIFF:
                comparison_y.append(0)
                comparison_X.append(vec1-vec2)
                comparison_y.append(1)
                continue
            comparison_y.append(v1 < v2)
    data = fit(comparison_X, comparison_y)
    column_name = f'{name} effect'
    
    df = pd.DataFrame(data=data, columns=['item', 'barcode', column_name])
    df['name'] = df.item.map(smart_inverse)
    df = df.sort_values(column_name)
    f = plt.figure(figsize=(10,7))
    p = sns.barplot(y='name', x=column_name, data=df.head(10).append(df.tail(10)))
order = "Jerusalem South Haifa|North Dan|Sharon|Central Tel"
importance_from_order(order, 'trig')

order = 'Jerusalem Sharon|South|North|Haifa Dan|Central Tel'
importance_from_order(order, 'hdl')

order = 'South Jerusalem|Sharon|North Dan|Central Tel|Haifa'
importance_from_order(order, 'transf')



# Attempts to predict socio/religion from sales

In [ ]:
model = Ridge(alpha=0.2)

city_embeddings = np.vstack([embeddings_dict[x] for x in stats_df.index.values])
city_embeddings_normalized = normalize(city_embeddings, axis=1)
for column in ['socio', 'haredim']:
    model.fit(city_embeddings_normalized, stats_df[column].values)
    top_products = [x.barcode for x in Dataset().products if x.usages > 10000]
    column_name = column+' effect'
    socio_df = pd.DataFrame(data=[[Dataset().barcode_to_product[x].name, x, model.predict(embeddings_dict[x].reshape(1, -1))[0] - model.intercept_] for x in top_products], columns=['item', 'barcode', column_name])
    
    socio_df['name'] = socio_df.item.map(smart_inverse)
    socio_df = socio_df.sort_values(column_name)
    f = plt.figure(figsize=(10,7))
    p = sns.barplot(y='name', x=column_name, data=socio_df.head(10).append(socio_df.tail(10)))
    p.set_title('Estimated ' + column +' correlation')
    print(column, 'R squared',model.score(city_embeddings_normalized, stats_df[column].values), 'MAE',   mean_squared_error(model.predict(city_embeddings_normalized), stats_df[column].values, squared=False))

In [ ]:
user_embeddings = np.vstack([embeddings_dict[x] for x in Users().active_users])
user_embeddings_normalized = normalize(user_embeddings, axis=1)
model.fit(city_embeddings, stats_df['socio'].values)
user_socio = model.predict(user_embeddings)
model.fit(city_embeddings, stats_df['haredim'].values)
user_religion = model.predict(user_embeddings)

In [ ]:
model = Ridge(alpha=0.2)
model.fit(city_embeddings, stats_df['socio'].values)
example_cities = list(stats_df.sort_values('n_users').index.values[-20:])
example_city_socio = stats_df.loc[example_cities]['socio'].values
city_users = [[x, smart_inverse(y), model.predict(embeddings_dict[x].reshape(1,-1))[0]]  for x,y in Users().active_users_cities.items() if y in example_cities]
df = pd.DataFrame(city_users, columns=['u_id', 'city', 'predicted_socio'])
plt.figure(figsize=(20,10))
p = sns.violinplot(x='city', y='predicted_socio', data=df, order=df.groupby('city')['predicted_socio'].mean().sort_values().index)
#for i in range(20):
    #p.plot([i-0.5,i+0.5], [v[i], v[i]])

In [ ]:
model = Ridge(alpha=0.2)
model.fit(city_embeddings, stats_df['socio'].values)
example_cities = list(stats_df.sort_values('n_users').index.values[-50:-40])+ list(stats_df.sort_values('n_users').index.values[-10:])
example_city_socio = stats_df.loc[example_cities]['socio'].values
city_users = [[x, smart_inverse(y), model.predict(embeddings_dict[x].reshape(1,-1))[0]]  for x,y in Users().active_users_cities.items() if y in example_cities]
df = pd.DataFrame(city_users, columns=['u_id', 'city', 'predicted_socio'])
plt.figure(figsize=(20,10))
p = sns.violinplot(x='city', y='predicted_socio', data=df, order=[smart_inverse(x) for x in example_cities])
for i in range(20):
    p.plot([i-0.5,i+0.5], [example_city_socio[i], example_city_socio[i]])

# User clustering and explaining

In [ ]:
def viz_silhoette(X, cluster_labels, title='', ax=None):
    if X.shape[0] > 10000:
        idx = np.random.choice(X.shape[0], size=10000, replace=False)
        X = X[idx]
        cluster_labels = cluster_labels[idx]
    
    n_clusters = len(set(cluster_labels))
    sample_silhouette_values = silhouette_samples(X, cluster_labels)
    
    if ax is None:
        f, ax1 = plt.subplots(1,1, figsize=(12,7))
    else:
        ax1 = ax
    ax1.set_ylim([0, X.shape[0] + (n_clusters + 1) * 10])
    y_lower = 10
    for i in range(n_clusters):
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values = sample_silhouette_values[cluster_labels == i]
        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i) / n_clusters)
        ax1.fill_betweenx(np.arange(y_lower, y_upper),
                          0, ith_cluster_silhouette_values,
                          facecolor=color, edgecolor=color, alpha=0.7)

        # Label the silhouette plots with their cluster numbers at the middle
        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples

    ax1.set_title(title)
    ax1.set_xlabel("The silhouette coefficient values. mean=" +str(np.mean(sample_silhouette_values)))
    ax1.set_ylabel("Cluster label")

    # The vertical line for average silhouette score of all the values
    ax1.axvline(x=np.mean(sample_silhouette_values), color="red", linestyle="--")

    ax1.set_yticks([])  # Clear the yaxis labels / ticks
    
    if ax is None:
        plt.show()


clf = MiniBatchKMeans(n_clusters=9, random_state=4)
#clf = GaussianMixture(n_components=5, random_state=3)
user_clusters = clf.fit_predict(user_embeddings_normalized)
viz_silhoette(user_embeddings_normalized, user_clusters)

In [ ]:
res = []
for k in range(9,11,2):
    for restart in range(5):
        clf = MiniBatchKMeans(n_clusters=k, random_state=restart)
    #clf = GaussianMixture(n_components=10, random_state=3)
        y = clf.fit_predict(user_embeddings_normalized)
        res.append([k, silhouette_score(user_embeddings_normalized, y)])
sns.scatterplot(x='n_clusters', y='silhoette', data=pd.DataFrame(data=res, columns=['n_clusters', 'silhoette']))

In [ ]:
%%time
LOAD_VECS = True
if LOAD_VECS:
    with open(CLUSTERING_PATH, 'rb') as f:
        user_vectors = pickle.load(f)
else:
    #user_vectors = {u: np.zeros(sum(LEVEL_SIZE[:-1])) for u in {x.user_id for x in transactions}}
    user_vectors = {u: np.zeros(Dataset().hierarchy_sizes[-2]) for u in Users().active_users}
    offset = sum(Dataset().hierarchy_sizes[:-2])
    for t in Dataset().transactions:
        if t.user_id not in user_vectors.keys():
            continue
        user_vectors[t.user_id][[Dataset().barcode_to_product[barcode].hierarchy_indices[3]-offset for barcode in t.barcodes]] += 1
        #user_vectors[t.user_id][p.hierarchy_indices[:min(p._representation+1, 4)]] += 1
    with open(CLUSTERING_PATH, 'wb') as f:
        pickle.dump(user_vectors, f)

tf_matrix = np.vstack([user_vectors[x] for x in Users().active_users])  
tf_normalized = normalize(tf_matrix, axis=1, norm='l2')

del user_vectors

In [ ]:
l3 = sorted([x for x in Dataset().index_of.keys() if x.startswith('3_')])
feature_names = [x[2:].replace('[', '(').replace(']', ')').replace('<', ' ') for x in l3]
df = pd.DataFrame(data=tf_normalized, columns=feature_names)
#

from sklearn.utils import resample
def split(cluster, np_array=False):
    X_train, X_test, y_train, y_test = train_test_split(df, [z==cluster for z in user_clusters], test_size=0.20, random_state=1)
    if X_train.shape[0] > sum(y_train)*2:
        z_df = X_train[[not x for x in y_train]]
        z_df = resample(z_df, n_samples=sum(y_train)*2, random_state=1)
        X_train = X_train[y_train].append(z_df)
        y_train = np.array([1]*(len(X_train) - len(z_df)) +[0]*len(z_df))
    # for the tests
    #X_train, _, y_train, _ = train_test_split(X_train, y_train, test_size=0.8, random_state=1)
    if np_array:
        return X_train.values, X_test.values, y_train, y_test
    return X_train, X_test, y_train, y_test

def viz_lr_mixed(cluster=0, k=20, positive=12):
    X_train, X_test, y_train, y_test = split(cluster, True)
    model = LR(max_iter=10000, penalty='l2', solver='liblinear', class_weight='balanced', C=100)
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    f1 = recall_score(y_test, predictions)
    positive_weights = X_train[y_train].mean(axis=0)*model.coef_[0]
    negative_weights = X_train[~y_train].mean(axis=0)*model.coef_[0]
    tmp = pd.DataFrame(sorted(list(zip(positive_weights, feature_names)), key=lambda x: x[0], reverse=True)[:positive], columns=['Logistic regression weight', 'sub-category'])
    tmp2 = pd.DataFrame(sorted(list(zip(negative_weights, feature_names)), key=lambda x: x[0], reverse=True)[-(k-positive):], columns=['Logistic regression weight', 'sub-category'])
    tmp = tmp.append(tmp2)
    tmp['sub-category'] = tmp['sub-category'].map(smart_inverse)
    f = plt.figure(figsize=(10,7))
    p = sns.barplot(y='sub-category', x='Logistic regression weight', data=tmp)
    
    socio = user_socio[y_train].mean()
    rel = user_religion[y_train].mean()
    p.set_title('Cluster {} with {:.1f}% of users. Recall: {:.1f}%. Socio status: {:.2f}. Predicted haredim rate: {:.2f}'.format(cluster, sum(y_train)/len(df)*100,f1*100, round(socio, 2), round(rel, 2)))
    plt.savefig('./img/'+str(cluster)+'.png')
    

In [ ]:
for i in range(9):
    viz_lr_mixed(i)

In [ ]:

def viz_explanation(cluster=0):
    X_train, X_test, y_train, y_test = split(cluster)
    hist = ClassHistogram().explain_data(X_train, y_train, name = 'Train Data')

    ebm = ExplainableBoostingClassifier(random_state=1, n_jobs=8, interactions=0, max_bins=30)
    ebm.fit(X_train, y_train)   #Works on dataframes and numpy arrays
    ebm_global = ebm.explain_global(name='EBM')
    ebm_local = ebm.explain_local(X_test[y_test][:5].append(X_test[y_test].mean(), ignore_index=True), [1]*6, name='EBM') #first 5 positive instances and cluster center as 6th one
    ebm_perf = ROC(ebm.predict_proba).explain_perf(X_test, y_test, name='EBM')
    lr = LogisticRegression(random_state=1, feature_names=feature_names, penalty='l1', solver='liblinear', C=1)
    lr.fit(X_train, y_train)
    #sorted(zip(lr.sk_model_.coef_[0], feature_names), key=lambda x: x[0], reverse=True)[:10]
    lr_perf = ROC(lr.predict_proba).explain_perf(X_test, y_test, name='Logistic Regression')

    lr_global = lr.explain_global(name='Logistic Regression')
    lr_local = lr.explain_local(X_test[y_test][:5].append(X_test[y_test].mean(), ignore_index=True), [1]*6, name='LR local' )
    
    show([hist, lr_global, lr_perf, ebm_global, ebm_perf, ebm_local, lr_local], share_tables=True)

def viz_shap(cluster=0):
    X_train, X_test, y_train, y_test = split(cluster, True)
    model = XGBClassifier(use_label_encoder=False) 
    #model = sklearn.linear_model.LogisticRegression(max_iter=100, penalty='l1', solver='liblinear')
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    print(cluster, accuracy_score(y_test, predictions), recall_score(y_test, predictions))
    X_train = pd.DataFrame(X_train)
    X_train.columns = [x[::-1] for x in feature_names] if DATASET == 'strauss' else feature_names
    explainer = shap.Explainer(model, X_train)
    shap.initjs()
    shap_values = explainer(X_train)
    shap.summary_plot(shap_values, X_train)
    
def viz_lr(cluster=0, viz_center=False, k=20, positive=15):
    X_train, X_test, y_train, y_test = split(cluster, True)
    model = LR(max_iter=100, penalty='l1', solver='liblinear')
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    print(accuracy_score(y_test, predictions), recall_score(y_test,predictions), precision_score(y_test, predictions))
    if viz_center:
        weights = X_train[y_train].mean(axis=0)*model.coef_[0]
    else:
        weights = model.coef_[0]
    named_coef = list(zip(weights, feature_names))
    tmp = pd.DataFrame(sorted(named_coef, key=lambda x: x[0], reverse=True), columns=['Logistic regression weight', 'sub-category'])
    return tmp.head(positive).append(tmp.tail(k-positive))

def create_lists(cluster=0):
    X_train, X_test, y_train, y_test = split(cluster)
    cluster_center = X_train.iloc[y_train].mean(axis=0)
    
    #ebm = ExplainableBoostingClassifier(random_state=1, n_jobs=8, interactions=0, max_bins=30)
    #ebm.fit(X_train, y_train)
    #ebm_list = []
    
    model = XGBClassifier(use_label_encoder=False) 
    model.fit(X_train, y_train)
    explainer = shap.TreeExplainer(model, X_train)
    shap_values = explainer(X_train.append(cluster_center, ignore_index=True))
    shap_aggregated = shap_values[:-1,:][y_train,:].mean(axis=0).values
    
    shap_center = shap_values[-1,:]
    
    lr = LR(max_iter=100, penalty='l1', solver='liblinear')
    lr.fit(X_train, y_train)
    lr_prediction = X_train.values[y_train].mean(axis=0)*lr.coef_[0]
    lr_weights = lr.coef_[0]
    
    return shap_aggregated, shap_center.values, lr_prediction, lr_weights

def rank_to_score(rank, l):
    return len(l)-rank-1

def aggregate_lists(lists, method='rank'):
    if method == 'score':
        stacked_scores = np.stack(lists).T
        norm_scores = sklearn.preprocessing.StandardScaler().fit_transform(stacked_scores)
        new_scores = norm_scores.mean(1)
        return new_scores
    
    scores = np.zeros(len(lists[0]))
    for l in lists:
        list_scores = Counter(np.abs(l))
        less_than = {sc: sum([v for k,v in list_scores.items() if k < sc ]) for sc in list_scores.keys()}
        for i, v in enumerate(np.abs(l)):
            scores[i] += less_than[v]
            
    for i in range(len(lists[0])):
        if all(l[i] >= 0 for l in lists):
            continue
        elif all(l[i] <= 0 for l in lists):
            scores[i] = -scores[i]
        else:
            scores[i]=0
    return scores

def top_k_features(scores, k=20, positive=15):
    best = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)
    best = [(feature_names[i], scores[i]) for i in best]
    return best[:positive] + best[-(k-positive):]
